#Data

In [3]:
%%writefile textrank.txt
3: BC−HurricaineGilbert, 09−11 339
4: BC−Hurricaine Gilbert, 0348
5: Hurricaine Gilbert heads toward Dominican Coast
6: By Ruddy Gonzalez
7: Associated Press Writer
8: Santo Domingo, Dominican Republic (AP)
9: Hurricaine Gilbert Swept towrd the Dominican Republic Sunday, and the Civil Defense alerted its heavily populated south coast to prepare for high winds, heavy rains, and high seas.
10: The storm was approaching from the southeast with sustained winds of 75 mph gusting to 92 mph.
11: "There is no need for alarm," Civil Defense Director Eugenio Cabral said in a television alert shortly after midnight Saturday.
12: Cabral said residents of the province of Barahona should closely follow Gilbert’s movement.
13: An estimated 100,000 people live in the province, including 70,000 in the city of Barahona, about 125 miles west of Santo Domingo.
14: Tropical storm Gilbert formed in the eastern Carribean and strenghtened into a hurricaine Saturday night.
15: The National Hurricaine Center in Miami reported its position at 2 a.m. Sunday at latitude 16.1 north, longitude 67.5 west, about 140 miles south of Ponce, Puerto Rico, and 200 miles southeast of Santo Domingo.
16: The National Weather Service in San Juan, Puerto Rico, said Gilbert was moving westard at 15 mph with a "broad area of cloudiness and heavy weather" rotating around the center of the storm.
17: The weather service issued a flash flood watch for Puerto Rico and the Virgin Islands until at least 6 p.m. Sunday.
18: Strong winds associated with the Gilbert brought coastal flooding, strong southeast winds, and up to 12 feet to Puerto Rico’s south coast.
19: There were no reports on casualties.
20: San Juan, on the north coast, had heavy rains and gusts Saturday, but they subsided during the night.
21: On Saturday, Hurricane Florence was downgraded to a tropical storm, and its remnants pushed inland from the U.S. Gulf Coast.
22: Residents returned home, happy to find little damage from 90 mph winds and sheets of rain.
23: Florence, the sixth named storm of the 1988 Atlantic storm season, was the second hurricane.
24: The first, Debby, reached minimal hurricane strength briefly before hitting the Mexican coast last month.

Overwriting textrank.txt


#Set enviroment variable for PySpark

In [4]:
import os
import sys
spark_home = os.environ['SPARK_HOME'] = '/opt/spark-1.5.0-bin-hadoop2.6/'
if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')

sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.8.2.1-src.zip'))
exec(open(os.path.join(spark_home,'python/pyspark/shell.py')).read())
##exec(open(os.path.join(spark_home,'bin/pyspark')).read())


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.5.0
      /_/

Using Python version 3.5.1 (default, Dec  7 2015 11:24:55)
SparkContext available as sc, HiveContext available as sqlContext.


In [ ]:
import os
import sys
spark_home = os.environ['SPARK_HOME'] = '/Users/jshanahan/Dropbox/Lectures-UC-Berkeley-ML-Class-2015/spark-1.5.0-bin-hadoop2.6/'
if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')
##sys.path.insert(0,os.path.join(spark_home,'python'))
##sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.8.2.1-src.zip'))
##execfile(os.path.join(spark_home,'python/pyspark/shell.py'))
execfile(os.path.join(spark_home,'bin/pyspark'))

#Text Rank Algo

In [9]:
from math import log

def computeContribs(sentences, rank):
    sumOfWeights = sum([s[1] for s in sentences])
    for sentence in sentences:
        yield (sentence[0], rank * sentence[1] / sumOfWeights)
# each record is (node, List of All Neighbors)        
def parseSentences(line):
    fields = line.split(':')
    return(fields[0], fields[1].replace(',','').split())

def generateLink(sentence1,sentence2):
    w = len(set(sentence1[1]) & set(sentence2[1]))/(log(len(sentence1[1]))+log(len(sentence2[1])))
    return(sentence1[0],(sentence2[0],w))

text = sc.textFile("./textrank.txt").map(parseSentences)
links = text.cartesian(text).filter(lambda s1,s2: s1[0]!=s2[0]) \
            .map(lambda s1,s2:generateLink(s1,s2)) \
            .groupByKey() \
            .filter(lambda Idx,links: sum([s[1] for s in links])>0) \
            .cache()
ranks = links.map(lambda url, neighbors: (url, 1.0))
for iteration in range(5):
    #Emit scores to all neighbor nodes 
    contribs = links.join(ranks).flatMap(
        lambda sentence, sentences, rank: computeContribs(sentences, rank))
    #sum up 
    ranks = contribs.reduceByKey(lambda x,y: x+y).mapValues(lambda rank: rank * 0.85 + 0.15)
ranks.collect()


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 10, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark-1.5.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/opt/spark-1.5.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark-1.5.0-bin-hadoop2.6/python/pyspark/rdd.py", line 2352, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark-1.5.0-bin-hadoop2.6/python/pyspark/rdd.py", line 2352, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark-1.5.0-bin-hadoop2.6/python/pyspark/rdd.py", line 317, i

In [6]:
links = text.cartesian(text).filter(lambda(s1,s2): s1[0]!=s2[0]).map(lambda(s1,s2):generateLink(s1,s2)).groupByKey().filter(lambda (Idx,links): sum([s[1] for s in links])>0).cache()
ranks = links.map(lambda (url, neighbors): (url, 1.0))
for iteration in xrange(5):
    #Emit scores to all neighbor nodes 
    contribs = links.join(ranks).flatMap(
        lambda (sentence, (sentences, rank)): computeContribs(sentences, rank))
    #sum up 
    ranks = contribs.reduceByKey(lambda x,y: x+y).mapValues(lambda rank: rank * 0.85 + 0.15)
print ranks.collect()

SyntaxError: invalid syntax (<ipython-input-6-74870e56c51c>, line 1)